In [1]:
import os
import sys
sys.path.append('/Users/cmagri1/OneDrive - Johns Hopkins/git')
from EncodingModel_cm296 import utils as emutils
from EncodingModel_cm296 import cv_regression as cv

# from EncodingModel_cm296 import feature_extractor as emfe
# from DataVisualizations_cm296 import scatterplots as emscatter
# from DataVisualizations_cm296 import utils as dvutils
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.decomposition import PCA
from tqdm import tqdm
from sklearn.model_selection import KFold

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
ROI = 'LOC'
Sub = 1

kf9 = KFold(n_splits = 9)
l2 = 0.0

In [3]:
layer =  {'conv_1', 'conv_2','conv_3','conv_4','conv_5'}

In [10]:
Keepncomps = [10,20,30,40,50,60,70,80]
# Keepncomps = [10,20]

In [5]:
keyword = {'ROIpred','DNNActvtn'}
#Load word2sense
#already preprocessed in MATLAB so that Wrd2Sns and THINGs overlap --> we have IMAGES, LABELS and SENSES
pathtofile = '../Code-001/'
features = pd.read_csv(pathtofile + "ThingsWrd2Sns.txt", sep=",")
features = features.values[:,1:features.shape[1]-1].astype(np.float)

In [6]:
def iter_cvregress(PCs,features,kf,keyword,ilayer,icomps):
    datapath = '/data-01/'
    print('k-fold regression, independet variable: ' + str(icomps) + ' PCs retained of ' + keyword + ' from ' + ilayer)
    rs = []#[[] for i in (0,(features.values.shape[1]-1))]
    for train_index, test_index in kf.split(PCs):
        train_features = PCs[train_index,]
        test_features =  PCs[test_index,]        
        train_Y = features[train_index,]
        test_Y = features[test_index,]
        _,r = cv.regression_iter(train_features, train_Y, test_features,  test_Y, l2=l2)
        rs.append(r)
    mean_r = np.nanmean(rs, axis=0) #TO handle Nans, since the feature space is so sparse
    #Saved information
    np.save(os.getcwd() + datapath + 'PredictSENSES_' + keyword + '_' + ilayer + '_'+ str(icomps) +'PCs', mean_r)
    return mean_r

In [7]:
def make_figure(mean_r,keyword,layer,icomps):
    figurepath = '/figures/'
    plt.figure(figsize=(100,35))
    # Set the font dictionaries (for plot title and axis titles)
    title_font = {'fontname':'Arial', 'size':'100', 'color':'black', 'weight':'normal',
  'verticalalignment':'bottom'} # Bottom vertical alignment for more space
    axis_font = {'fontname':'Arial', 'size':'100'}
    sns.set()
    n= len(~np.isnan(mean_r))-1
    mean_r_nonan = mean_r[~np.isnan(mean_r)]
    
    ax = plt.subplot() # Defines ax variable by creating an empty plot
    
    # Set the tick labels font
    for label in (ax.get_xticklabels() + ax.get_yticklabels()):
        label.set_fontname('Arial')
        label.set_fontsize(100)
    
    plt.xlabel('Sense', **axis_font)
    plt.ylabel('PredictionAccuracy', **axis_font)
    
    plt.title('PredictSENSES ' + keyword + ' ' + layer + ' '+ str(icomps) +'PCs', **title_font)
    
    #Plot the Image!
    # plt.bar(range(0,n), np.sort(mean_r[0:n]), color = 'darkred')
    plt.bar(range(0,n), np.sort(mean_r_nonan)[::-1], color = 'darkred')  
    
    #ylim
    plt.ylim(-0.2, 0.6) 
    
    #save figure
    plt.savefig(os.getcwd() + figurepath + 'PredictSENSES_' + keyword + '_' + layer + '_'+ str(icomps) +'PCs.png')
    sns.reset_defaults()

In [ ]:
for ilayer in layer:
    for ikeyword in keyword:        
        if ikeyword is 'ROIpred':
            #load ROIpred as predictor variable
            predictor_variable = np.load(os.getcwd() + '/data/' +  "ROIpred_Sub" + str(Sub) + '_' + ROI + "_" + ilayer + '.npy')
        elif ikeyword is 'DNNActvtn':
            predictor_variable = pd.read_csv(os.getcwd() + '/data/' +  "things_" + ilayer + '.csv').iloc[:,1:].to_numpy()
        
        
        for icomps in Keepncomps:
            mean_r = {}
            
            pca = PCA(n_components=icomps)
            PCs = pca.fit_transform(predictor_variable)
            
            mean_r = iter_cvregress(PCs,features,kf9,ikeyword,ilayer,icomps)
            make_figure(mean_r,ikeyword,ilayer,icomps)

k-fold regression, independet variable: 10 PCs retained of DNNActvtn from conv_5


/Users/cmagri1/OneDrive - Johns Hopkins/git/EncodingModel_cm296/cv_regression.py:44: RuntimeWarning: invalid value encountered in true_divide
  p_coef = numerator/denominator
/Users/cmagri1/OneDrive - Johns Hopkins/git/EncodingModel_cm296/cv_regression.py:44: RuntimeWarning: invalid value encountered in true_divide
  p_coef = numerator/denominator
/Users/cmagri1/OneDrive - Johns Hopkins/git/EncodingModel_cm296/cv_regression.py:44: RuntimeWarning: invalid value encountered in true_divide
  p_coef = numerator/denominator
/Users/cmagri1/OneDrive - Johns Hopkins/git/EncodingModel_cm296/cv_regression.py:44: RuntimeWarning: invalid value encountered in true_divide
  p_coef = numerator/denominator
/Users/cmagri1/OneDrive - Johns Hopkins/git/EncodingModel_cm296/cv_regression.py:44: RuntimeWarning: invalid value encountered in true_divide
  p_coef = numerator/denominator
/Users/cmagri1/OneDrive - Johns Hopkins/git/EncodingModel_cm296/cv_regression.py:44: RuntimeWarning: invalid value encountere

k-fold regression, independet variable: 20 PCs retained of DNNActvtn from conv_5


/Users/cmagri1/OneDrive - Johns Hopkins/git/EncodingModel_cm296/cv_regression.py:44: RuntimeWarning: invalid value encountered in true_divide
  p_coef = numerator/denominator
/Users/cmagri1/OneDrive - Johns Hopkins/git/EncodingModel_cm296/cv_regression.py:44: RuntimeWarning: invalid value encountered in true_divide
  p_coef = numerator/denominator
/Users/cmagri1/OneDrive - Johns Hopkins/git/EncodingModel_cm296/cv_regression.py:44: RuntimeWarning: invalid value encountered in true_divide
  p_coef = numerator/denominator
/Users/cmagri1/OneDrive - Johns Hopkins/git/EncodingModel_cm296/cv_regression.py:44: RuntimeWarning: invalid value encountered in true_divide
  p_coef = numerator/denominator
/Users/cmagri1/OneDrive - Johns Hopkins/git/EncodingModel_cm296/cv_regression.py:44: RuntimeWarning: invalid value encountered in true_divide
  p_coef = numerator/denominator
/Users/cmagri1/OneDrive - Johns Hopkins/git/EncodingModel_cm296/cv_regression.py:44: RuntimeWarning: invalid value encountere

k-fold regression, independet variable: 30 PCs retained of DNNActvtn from conv_5


...

/Users/cmagri1/OneDrive - Johns Hopkins/git/EncodingModel_cm296/cv_regression.py:44: RuntimeWarning: invalid value encountered in true_divide
  p_coef = numerator/denominator
/Users/cmagri1/OneDrive - Johns Hopkins/git/EncodingModel_cm296/cv_regression.py:44: RuntimeWarning: invalid value encountered in true_divide
  p_coef = numerator/denominator
/Users/cmagri1/OneDrive - Johns Hopkins/git/EncodingModel_cm296/cv_regression.py:44: RuntimeWarning: invalid value encountered in true_divide
  p_coef = numerator/denominator
/Users/cmagri1/OneDrive - Johns Hopkins/git/EncodingModel_cm296/cv_regression.py:44: RuntimeWarning: invalid value encountered in true_divide
  p_coef = numerator/denominator
/Users/cmagri1/OneDrive - Johns Hopkins/git/EncodingModel_cm296/cv_regression.py:44: RuntimeWarning: invalid value encountered in true_divide
  p_coef = numerator/denominator
/Users/cmagri1/OneDrive - Johns Hopkins/git/EncodingModel_cm296/cv_regression.py:44: RuntimeWarning: invalid value encountere

k-fold regression, independet variable: 70 PCs retained of ROIpred from conv_4


/Users/cmagri1/OneDrive - Johns Hopkins/git/EncodingModel_cm296/cv_regression.py:44: RuntimeWarning: invalid value encountered in true_divide
  p_coef = numerator/denominator
/Users/cmagri1/OneDrive - Johns Hopkins/git/EncodingModel_cm296/cv_regression.py:44: RuntimeWarning: invalid value encountered in true_divide
  p_coef = numerator/denominator
/Users/cmagri1/OneDrive - Johns Hopkins/git/EncodingModel_cm296/cv_regression.py:44: RuntimeWarning: invalid value encountered in true_divide
  p_coef = numerator/denominator
/Users/cmagri1/OneDrive - Johns Hopkins/git/EncodingModel_cm296/cv_regression.py:44: RuntimeWarning: invalid value encountered in true_divide
  p_coef = numerator/denominator
/Users/cmagri1/OneDrive - Johns Hopkins/git/EncodingModel_cm296/cv_regression.py:44: RuntimeWarning: invalid value encountered in true_divide
  p_coef = numerator/denominator
/Users/cmagri1/OneDrive - Johns Hopkins/git/EncodingModel_cm296/cv_regression.py:44: RuntimeWarning: invalid value encountere

k-fold regression, independet variable: 80 PCs retained of ROIpred from conv_4


/Users/cmagri1/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/ridge.py:147: LinAlgWarning: Ill-conditioned matrix (rcond=9.51327e-17): result may not be accurate.
  overwrite_a=True).T
/Users/cmagri1/OneDrive - Johns Hopkins/git/EncodingModel_cm296/cv_regression.py:44: RuntimeWarning: invalid value encountered in true_divide
  p_coef = numerator/denominator
/Users/cmagri1/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/ridge.py:147: LinAlgWarning: Ill-conditioned matrix (rcond=9.86402e-17): result may not be accurate.
  overwrite_a=True).T
/Users/cmagri1/OneDrive - Johns Hopkins/git/EncodingModel_cm296/cv_regression.py:44: RuntimeWarning: invalid value encountered in true_divide
  p_coef = numerator/denominator
/Users/cmagri1/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/ridge.py:147: LinAlgWarning: Ill-conditioned matrix (rcond=9.87077e-17): result may not be accurate.
  overwrite_a=True).T
/Users/cmagri1/OneDrive - Johns Hopkins/git/Enc